# Web Scraping

Import the libraries.

In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

Open the Browser.

In [11]:
driver = webdriver.Edge()

Navigate to the event page.

In [12]:
driver.get("https://chess.com/events/2023-bullet-chess-championship-main-event/games")

WebDriverException: Message: target frame detached
  (failed to check if window was closed: disconnected: Unable to receive message from renderer)
  (Session info: MicrosoftEdge=117.0.2045.60)
Stacktrace:
	GetHandleVerifier [0x00007FF68FD599F2+63682]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF68FCE1942+267762]
	(No symbol) [0x00007FF68FA9AA40]
	(No symbol) [0x00007FF68FA84005]
	(No symbol) [0x00007FF68FA8235B]
	(No symbol) [0x00007FF68FA8326E]
	(No symbol) [0x00007FF68FA90F5F]
	(No symbol) [0x00007FF68FAA8159]
	(No symbol) [0x00007FF68FAADD9A]
	(No symbol) [0x00007FF68FA83BC6]
	(No symbol) [0x00007FF68FA8379D]
	(No symbol) [0x00007FF68FAA7D2B]
	(No symbol) [0x00007FF68FB26DE2]
	(No symbol) [0x00007FF68FB0C623]
	(No symbol) [0x00007FF68FADDA7A]
	(No symbol) [0x00007FF68FADCD6B]
	(No symbol) [0x00007FF68FADE204]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF68FF25EF9+1233737]
	(No symbol) [0x00007FF68FB5B274]
	Microsoft::Applications::

Before going any further, it's necessary a scroll-down of the page to load more matches.
Then, get every element having the 'round-games-list-item-games' class.

In [ ]:
game_items = driver.find_elements(By.CLASS_NAME, "round-games-list-item-games")

Now, extract all the links to the games.

In [ ]:
# Create a new empty list
links = []

# For each game_item, find every elements 'a' with href attribute
for game_item in game_items:
    game_links = game_item.find_elements(by=By.TAG_NAME, value="a")

    # Add all links to the list
    for game_link in game_links:
        links.append(game_link.get_attribute("href"))

# Check if it loaded enough matches
if len(links) > 31:
    links = links[:32]
    print('Success')
else:
    print('Load more matches')

Success


## Extract informations

In [ ]:
# Variables for summary Dataframe
player = []
color_generic = []
clock_value = []
clock_result = []
accuracy_score = []
game = []

# Variables for in-game specifics Dataframe
move = []
skill = []
turn = []
color = []
match = []
n = 0


for link in links:
    # Load the page
    driver.get(link)
    sleep(5)
    print(link)
    

    # Save the player ...
    if 'Nakamura_Hikaru-Carlsen_Magnus' in link:
        player.append('Hikaru Nakamura')
        player.append('Magnus Carlsen')
    elif 'Carlsen_Magnus-Nakamura_Hikaru' in link:
        player.append('Magnus Carlsen')
        player.append('Hikaru Nakamura')
    # ... and the piece colors
    color_generic.append('W')
    color_generic.append('B')

    # Saving clock-values and results
    for i in range(1, -1, -1):
        clock_value.append(driver.find_elements(By.CLASS_NAME, 'clock-value')[i].text)
        clock_result.append(driver.find_elements(By.CLASS_NAME, 'clock-result')[i].text)
    
    # Saving the accuracy
    for i in range(2):
        accuracy_score.append(driver.find_elements(By.CLASS_NAME, 'accuracy-score-value')[i].text)
    
    # Then save the game
    n += 1
    game.append(n)
    game.append(n)
    

    # Load the match-analysis tab
    driver.find_element(By.CSS_SELECTOR, "button[data-cy='game-sidebar-tab-analysis']").click()
    sleep(3)

    # Takes moves in the game analysis
    moves_driver = driver.find_elements(By.CLASS_NAME, 'move-node')
    moves = []
    for m in moves_driver:
        moves.append(m.find_element(By.CLASS_NAME, 'move-text').text)
        # Check if the move is 'bad/good' or 'normal'
        if 'customColor' in m.get_attribute('class'):
            # Get style
            style = m.get_attribute('style')
            index = style.find('#')
            skill.append(style[index:index + 7])
        else:
            skill.append('=')
    
    # Storing all moves
    move += moves

    # Saving the colors and the number of the game
    for i in range(len(moves)):
        turn.append(int(i/2) + 1)
        if i % 2 == 0:
            color.append('W')
        else:
            color.append('B')
        match.append(n)
    
    print('Game', n, 'fatto')

https://www.chess.com/events/2023-bullet-chess-championship-main-event/05-01/Nakamura_Hikaru-Carlsen_Magnus
ciaone
Game 1 fatto
https://www.chess.com/events/2023-bullet-chess-championship-main-event/05-02/Carlsen_Magnus-Nakamura_Hikaru
ciao
Game 2 fatto
https://www.chess.com/events/2023-bullet-chess-championship-main-event/05-03/Nakamura_Hikaru-Carlsen_Magnus
ciaone


KeyboardInterrupt: 

In [ ]:
print(len(player))
print(len(color_generic))
print(len(clock_value))
print(len(clock_result))
print(len(accuracy_score))
print(len(game))

print(len(move))
print(len(skill))
print(len(turn))
print(len(color))
print(len(match))

0
64
64
64
64
64
3439
3439
3439
3439
3439


In [ ]:
df_summary = pd.DataFrame({
    'player': player,
    'color_generic': color_generic,
    'clock_value': clock_value,
    'clock_result': clock_result,
    'accuracy_score': accuracy_score,
    'game': game
})

df_summary

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,Hikaru Nakamura,W,0:05.1,DRAW,80.4,2
3,Magnus Carlsen,B,0:05.1,DRAW,78.2,2
4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...
59,Magnus Carlsen,B,0:17.5,LOST,82.7,30
60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,Hikaru Nakamura,W,0:09.8,LOST,92.4,32


In [ ]:
df_games = pd.DataFrame({
    'move': move,
    'skill': skill,
    'turn': turn,
    'color': color,
    'match': match
})

df_games

,move,skill,turn,color,match
0,e4,=,1,W,1
1,e5,=,1,B,1
2,Nf3,=,2,W,1
3,Nc6,=,2,B,1
4,d4,=,3,W,1
...,...,...,...,...,...
3434,Qd2+,=,56,B,32
3435,Kf2,=,57,W,32
3436,c2,=,57,B,32
3437,Qc7,=,58,W,32


In [ ]:
df_games.skill = df_games.skill.str.strip()

In [ ]:
df_summary.to_csv('summary.csv', index=False)
df_games.to_csv('games.csv', index=False)

In [ ]:
df_games.skill.value_counts()

skill
=          3058
#e6912c     141
#5c8bb0     130
#ee6b55      81
#b33430      25
#1baca6       4
Name: count, dtype: int64

In [ ]:
df_games[df_games.skill == '#1baca6']

,move,skill,turn,color,match
1166,Bxh7+,#1baca6,23,W,11
1607,Nxg6,#1baca6,31,W,16
2173,Nxd5,#1baca6,18,B,21
3406,Qxe4,#1baca6,42,B,32


In [ ]:
symbol = ['??', '?', 'X', '!', '!!']
meaning = ['Blunder', 'Mistake', 'Missed win', 'Good move', 'Brilliant move']
hex = ['#b33430', '#e6912c', '#ee6b55', '#5c8bb0', '#1baca6']

df_annotation = pd.DataFrame({
    'symbol': symbol,
    'meaning': meaning,
    'hex': hex
})

df_annotation

,symbol,meaning,hex
0,??,Blunder,#b33430
1,?,Mistake,#e6912c
2,X,Missed win,#ee6b55
3,!,Good move,#5c8bb0
4,!!,Brilliant move,#1baca6


In [ ]:
df_annotation.to_csv('data/annotation.csv')

## Dati da sistemare
A volte il computer/browser è troppo lento a caricare, di conseguenza alcuni valori di Accuracy sono sbagliati.

In [ ]:
df_sistemaggio = pd.read_csv('data/summary.csv')

df_sistemaggio

,Unnamed: 0,player,color_generic,clock_value,clock_result,accuracy_score,game
0,0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,2,Magnus Carlsen,W,0:05.1,DRAW,80.4,2
3,3,Hikaru Nakamura,B,0:05.1,DRAW,78.2,2
4,4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...,...
59,59,Hikaru Nakamura,B,0:17.5,LOST,82.7,30
60,60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,62,Magnus Carlsen,W,0:09.8,LOST,92.4,32


In [ ]:
p = []
for i in range(int(len(df_sistemaggio)/2)):
    if i % 2 == 1:
        p.append('Magnus Carlsen')
        p.append('Hikaru Nakamura')
    else:
        p.append('Hikaru Nakamura')
        p.append('Magnus Carlsen')

df_sistemaggio.player = p
df_sistemaggio

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,Magnus Carlsen,W,0:05.1,DRAW,80.4,2
3,Hikaru Nakamura,B,0:05.1,DRAW,78.2,2
4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...
59,Hikaru Nakamura,B,0:17.5,LOST,82.7,30
60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,Magnus Carlsen,W,0:09.8,LOST,92.4,32


In [ ]:
df_sistemaggio.to_csv('data/summary.csv')

In [ ]:
df_sistemaggio[df_sistemaggio.accuracy_score == 100]

,Unnamed: 0,player,color_generic,clock_value,clock_result,accuracy_score,game
6,6,Magnus Carlsen,W,0:04.4,WON,100.0,4
7,7,Hikaru Nakamura,B,0:02.3,LOST,100.0,4
12,12,Hikaru Nakamura,W,0:12.5,DRAW,100.0,7
13,13,Magnus Carlsen,B,0:20,DRAW,100.0,7
40,40,Hikaru Nakamura,W,0:06.0,DRAW,100.0,21
41,41,Magnus Carlsen,B,0:05.0,DRAW,100.0,21


In [ ]:
indexes = df_sistemaggio[df_sistemaggio.accuracy_score == 100].index
df_sistemaggio['accuracy_score'].iloc[indexes[0]] = 83.0
df_sistemaggio['accuracy_score'].iloc[indexes[1]] = 76.1
df_sistemaggio['accuracy_score'].iloc[indexes[2]] = 93.4
df_sistemaggio['accuracy_score'].iloc[indexes[3]] = 94.1
df_sistemaggio['accuracy_score'].iloc[indexes[4]] = 90.8
df_sistemaggio['accuracy_score'].iloc[indexes[5]] = 91.5

C:\Users\Nicola\AppData\Local\Temp\ipykernel_4168\1872921699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sistemaggio['accuracy_score'].iloc[indexes[0]] = 83.0
C:\Users\Nicola\AppData\Local\Temp\ipykernel_4168\1872921699.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sistemaggio['accuracy_score'].iloc[indexes[1]] = 76.1
C:\Users\Nicola\AppData\Local\Temp\ipykernel_4168\1872921699.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [ ]:
df_sistemaggio.iloc[[6,7,12,13,40,41]]

,Unnamed: 0,player,color_generic,clock_value,clock_result,accuracy_score,game
6,6,Magnus Carlsen,W,0:04.4,WON,83.0,4
7,7,Hikaru Nakamura,B,0:02.3,LOST,76.1,4
12,12,Hikaru Nakamura,W,0:12.5,DRAW,93.4,7
13,13,Magnus Carlsen,B,0:20,DRAW,94.1,7
40,40,Hikaru Nakamura,W,0:06.0,DRAW,90.8,21
41,41,Magnus Carlsen,B,0:05.0,DRAW,91.5,21


In [ ]:
print('Average accuracy of Magnus Carlsen: ', df_sistemaggio[df_sistemaggio['player'] == 'Magnus Carlsen']['accuracy_score'].mean())
print('Average accuracy of Hikaru Nakamura: ', df_sistemaggio[df_sistemaggio['player'] == 'Hikaru Nakamura']['accuracy_score'].mean())

Average accuracy of Magnus Carlsen:  87.63125
Average accuracy of Hikaru Nakamura:  87.225


# Unione dei csv

In [ ]:
df_games = pd.read_csv('data/games.csv')
df_games.head()

,move,skill,turn,color,match
0,e4,=,1,W,1
1,e5,=,1,B,1
2,Nf3,=,2,W,1
3,Nc6,=,2,B,1
4,d4,=,3,W,1


In [ ]:
df_summary = pd.read_csv('data/summary.csv', delimiter=';')
df_summary.head()

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,961,1
1,Magnus Carlsen,B,0:25,DRAW,960,1
2,Magnus Carlsen,W,0:05.1,DRAW,804,2
3,Hikaru Nakamura,B,0:05.1,DRAW,782,2
4,Hikaru Nakamura,W,0:37,WON,922,3


### Ma prima... estraiamo i "take"

In [25]:
df_games[df_games['move'].str.contains('x')]

,move,skill,turn,color,match,piece_that_take,piece_taken
5,exd4,=,3,B,1,e,d
6,Nxd4,=,4,W,1,N,d
10,Nxc6,=,6,W,1,N,c
11,bxc6,=,6,B,1,b,c
16,exd5,=,9,W,1,e,d
...,...,...,...,...,...,...,...
3406,Qxe4,#1baca6,42,B,32,Q,e
3407,Qxb7,=,43,W,32,Q,b
3414,Nxh3+,=,46,B,32,N,h+
3415,gxh3,=,47,W,32,g,h


In [ ]:
split_moves = df_games['move'].str.split('x', 1, expand=True)

df_games['piece_that_take'] = split_moves[0]
df_games['piece_taken'] = split_moves[1]

C:\Users\Nicola\AppData\Local\Temp\ipykernel_9348\1262988518.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  split_moves = df_games['move'].str.split('x', 1, expand=True)


In [20]:
df_games['piece_that_take'] = [move.split('x')[0] if 'x' in move else None for move in df_games['move']]

In [24]:
df_games

,move,skill,turn,color,match,piece_that_take,piece_taken
0,e4,=,1,W,1,None,None
1,e5,=,1,B,1,None,None
2,Nf3,=,2,W,1,None,None
3,Nc6,=,2,B,1,None,None
4,d4,=,3,W,1,None,None
...,...,...,...,...,...,...,...
3434,Qd2+,=,56,B,32,None,None
3435,Kf2,=,57,W,32,None,None
3436,c2,=,57,B,32,None,None
3437,Qc7,=,58,W,32,None,None


In [23]:
df_games['piece_that_take'] = df_games['piece_that_take'].str.replace('[0-9]', '')

df_games['piece_taken'] = df_games['piece_taken'].str.replace('[0-9]', '')
df_games['piece_that_take'] = df_games['piece_that_take'].str.replace('[+-]', '')

C:\Users\Nicola\AppData\Local\Temp\ipykernel_9348\3718230390.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_games['piece_that_take'] = df_games['piece_that_take'].str.replace('[0-9]', '')
C:\Users\Nicola\AppData\Local\Temp\ipykernel_9348\3718230390.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_games['piece_that_take'] = df_games['piece_that_take'].str.replace('[+-]', '')
C:\Users\Nicola\AppData\Local\Temp\ipykernel_9348\3718230390.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_games['piece_taken'] = df_games['piece_taken'].str.replace('[0-9]', '')


In [27]:
df_games.to_csv('data/games_modified.csv')